Copyright (c) Microsoft Corporation.<br>
Licensed under the MIT License.

# 4. OpenVINO Conversion

IMPORTANT:  The conversion command within this notebook is to be run with Intel's OpenVINO Toolkit docker container.

In this notebook we will:
- Convert the TensorFlow model to OpenVINO format

## Prerequisites
- Trained TensorFlow model (frozen graph format) downloaded from the experiment from following `3.Train_with_AzureML.ipynb`

More information on OpenVINO toolkit installation can be found at [install and set up the OpenVINO Toolkit](https://docs.openvinotoolkit.org/latest/installation_guides.html).

## Convert the model from frozen graph to an intermediate representation and then to a blob format

The following script to be run in the docker container, will convert the TensorFlow frozen graph to the OpenVINO IR format and then to `blob`.

In [ ]:
%%writefile experiment_outputs/compile.sh
#!/bin/bash

# OpenVINO compilation script

cd experiment_outputs

source /opt/intel/openvino_2021/bin/setupvars.sh

python3 /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_tf.py \
    --input_model frozen_inference_graph.pb \
    --tensorflow_object_detection_api_pipeline_config ../project_files/ssdlite_mobilenet_retrained.config \
    --transformations_config \
    /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json \
    --reverse_input_channels > openvino_log1.txt

/opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m \
    frozen_inference_graph.xml \
    -o ssdlite_mobilenet_v2.blob \
    -VPU_NUMBER_OF_SHAVES 8 \
    -VPU_NUMBER_OF_CMX_SLICES 8 -ip U8 -op FP32 > openvino_log2.txt

Here we will use the OpenVINO to leverage the OpenVINO model converters and optimizer.  You may need to replace `$(pwd)` with your current working directory path.  The `xLinkUsb` error in the logs file is expected and should be ignored.

In [ ]:
! docker run --rm --privileged -v $(pwd):/working -w /working openvino/ubuntu18_dev:2021.1 bash experiment_outputs/compile.sh